# Problem 1

In [44]:
import numpy as np

def fft(x):
#     assert len(x) & (len(x) - 1) == 0, "len of vector isn't power of 2"
    def fft(x, n):
        if n <= 128:
            i, j = np.meshgrid(np.arange(n), np.arange(n))
            omega = np.exp( - 2 * np.pi * 1j / n)
            W = np.power(omega, i * j)
            return W.dot(x)
        n >>= 1
        even = np.empty(n).astype(np.complex)
        odd = np.empty(n).astype(np.complex)
        for i in range(n):
            even[i] = x[i >> 1]
            odd[i] = x[(i >> 1) & 1]
        even_res = fft(even, n)
        odd_res = fft(odd, n)
        odd_res *= np.fromfunction(lambda k: np.exp(-np.pi * 1j * k / n), [n])
        return np.append(even_res + odd_res, even_res - odd_res)
    
    n = x.size
    return fft(x.astype(np.complex), n)

In [ ]:
n = 1 << 15
x = np.random.rand(n)
# print np.linalg.norm(fft(x) - np.fft.fft(x))
res1 = %timeit -o fft(x)
res2 = %timeit -o np.fft.fft(x)
print res1.best / res2.best

In [99]:
np.exp(-2 * np.pi * 1j * 10 / 107)

(0.83248770674817829-0.55404351644357253j)

# Problem 2